In [1]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import json
import sys

In [2]:
url = "https://docs.oracle.com/en/cloud/saas/netsuite/ns-online-help/toc.htm"
res = requests.get(url)
print(res.status_code)

200


In [3]:
sp = soup(res.text, "lxml")

In [4]:
def return_tree(item, tags):
    item1 = item.find_all("li")
    for values in item1:
        _values = values.find_all("li")
        if len(_values) > 0:
            _tags = return_tree(values, {})
            name = values.find("a")
            tags[name.text] = {"name": name.text, "value": name, "sublist": _tags}
        else:
            name = values.find("a")
            tags[name.text] = {"name": name.text, "value": name, "sublist": []}
#             tags.append(values.text)
    return tags
tags = return_tree(sp.find("ul"), {})

In [5]:
from copy import deepcopy
def remove_repeative(tags):
    for item in tags.keys():
        if tags[item] and tags[item]["sublist"]:
            sublist = tags[item]["sublist"]
            for item1 in sublist:
                if sublist[item1] and sublist[item1]["sublist"]:
                    if item1 in tags:
                        tags[item1] = ""
                    tags[item]["sublist"] = remove_repeative(tags[item]["sublist"].copy())
                else:
                    if item1 in tags:
                        tags[item1] = ""
    return tags

preprocessed_tags = remove_repeative(tags)          

In [8]:
preprocessed_tags["What's New"]

{'name': "What's New",
 'value': <a href="set_N20140200.html">What's New</a>,
 'sublist': {'Sneak Peeks': {'name': 'Sneak Peeks',
   'value': <a href="chapter_N3859207.html">Sneak Peeks</a>,
   'sublist': []},
  'Release Notes': {'name': 'Release Notes',
   'value': <a href="book_N3865324.html">Release Notes</a>,
   'sublist': {'NetSuite 2022.2 Release Notes': {'name': 'NetSuite 2022.2 Release Notes',
     'value': <a href="chapter_N3944673.html">NetSuite 2022.2 Release Notes</a>,
     'sublist': {'Accounting': {'name': 'Accounting',
       'value': <a href="section_4316104403.html">Accounting</a>,
       'sublist': []},
      'Account Setup and Maintenance': {'name': 'Account Setup and Maintenance',
       'value': <a href="section_158921855128.html">Account Setup and Maintenance</a>,
       'sublist': []},
      'Authentication': {'name': 'Authentication',
       'value': <a href="section_158921905537.html">Authentication</a>,
       'sublist': []},
      'Banking': {'name': 'Banking

In [ ]:
df = pd.DataFrame(columns=['name', 'html_page_link', 'tags'])
def generate_html_page(tags, tag_names=[]):
    global df
    for item in tags.keys():
        if tags[item] and tags[item]["sublist"]:
            df = df.append({"name": tags[item]["name"], "html_page_link": tags[item]["value"]["href"],
                                     "tags": tag_names + [item]}, ignore_index=True)
            sublist = tags[item]["sublist"]
            for item1 in sublist:
                if sublist[item1] and sublist[item1]["sublist"]:
                    generate_html_page(tags[item]["sublist"].copy(), tag_names + [item])
                if sublist[item1]:
                    df = df.append({"name": sublist[item1]["name"], "html_page_link": sublist[item1]["value"]["href"],
                                         "tags": tag_names + [item]}, ignore_index=True)
    return values
values = generate_html_page(preprocessed_tags, [])

In [21]:
df = pd.DataFrame(columns=['name', 'html_page_link', 'tags'])
def generate_dataframe(tags, tag_names=[]):
    global df
    for item in tags.keys():
        if tags[item]:
            if "value" in tags[item]:
#                 print(tags[item]["value"])
                df = df.append({"name": tags[item]["name"], "html_page_link": tags[item]["value"]["href"],
                                     "tags": tag_names + [item]}, ignore_index=True)
            if "sublist" in tags[item] and tags[item]["sublist"]:
                generate_dataframe(tags[item]["sublist"], tag_names= tag_names + [tags[item]["name"]])
generate_dataframe(preprocessed_tags)

In [26]:
df.to_csv("data.csv")

In [49]:
def process_text(text):
    text = text.strip()
    if "\n" in text:
        text = text.replace("\n", "")
    return text

In [111]:
def extract_text_from_tags(tags_text):
    if len(tags_text) == 1:
        return process_text(tags_text[0].text)
    else:
        text = ""
        for item in tags_text:
#             print(item)
            if "href" in item:
#                 print(item["href"])
                text = " " + text 
#             print("--------",item.text, text)
            _text = process_text(item.text)
#             print("----&&&&&&&&", _text)
            text += " " + _text
        return text

In [118]:
base_url = "https://docs.oracle.com/en/cloud/saas/netsuite/ns-online-help/"
text_data = []
for item in df.iterrows():
    print(item[1]["html_page_link"])
    res = requests.get(base_url + item[1]["html_page_link"])
    sp = soup(res.text, "lxml")
    text = ""
    spans = sp.find_all("span")
    text = extract_text_from_tags(spans)
    paragraphs = sp.find_all("p")
#     print(paragraphs)
    para_text = extract_text_from_tags(paragraphs)
#     print("para_text", para_text)
    if para_text:
        text += " " + para_text
    text_data.append(text)
#     print(text)

set_N20140200.html
chapter_N3859207.html
book_N3865324.html
chapter_N3944673.html
section_4316104403.html
section_158921855128.html
section_158921905537.html
section_4316105394.html
section_162620661407.html
section_160418265800.html
section_N3945981.html
section_161833073156.html
section_4316104783.html
section_158921892460.html
section_1508852924.html
section_4322136086.html
section_1529087731.html
section_156413618375.html
section_4316104540.html
section_3715561461.html
section_N3949039.html
section_N3945762.html
section_1507562099.html
section_1508778599.html
section_157913557426.html
section_1508778865.html
section_1508778807.html
section_1521115850.html
section_1508932552.html
section_1508778720.html
section_1508779004.html
section_1508778763.html
section_N3946132.html
section_1558730192.html
section_N3949604.html
section_N3950559.html
section_4316106593.html
section_N3945807.html
chapter_4410490401.html
section_160285387367.html
section_165338310610.html
section_165338276588.htm

article_159127266292.html
article_159127287838.html
article_159127297984.html
article_159127305801.html
article_159067372130.html
article_159067389515.html
article_158142795285.html
section_157598981990.html
subsect_1522434159.html
subsect_1522768517.html
section_1519995046.html
article_161046107399.html
article_161046137484.html
section_1522766078.html
article_158945626309.html
section_159594525073.html
section_156112187780.html
article_159594230761.html
section_156016689486.html
section_156034925776.html
article_159620404395.html
article_159465073582.html
section_156034956286.html
chapter_N1486105.html
section_N1486393.html
section_N1486610.html
section_N1486928.html
section_N1487157.html
bridgehead_N1487235.html
bridgehead_N1488569.html
bridgehead_N1489083.html
bridgehead_N1489533.html
bridgehead_4308408763.html
section_N1489768.html
section_N1490202.html
section_N1490669.html
section_N1492223.html
section_N1492389.html
section_N1492480.html
section_159249226797.html
section_1592492

subsect_1525368686.html
subsect_1518817923.html
subsect_1518813066.html
subsect_1518211824.html
section_1520981877.html
bridgehead_4430963939.html
subsect_1510866606.html
subsect_1516796571.html
bridgehead_3715154860.html
section_N3827717.html
section_1541168340.html
chapter_156399873235.html
section_156477805451.html
section_156399898626.html
section_156503115372.html
section_156503101193.html
section_156503088100.html
section_156503080375.html
section_156399919135.html
section_156399923739.html
section_158025750790.html
subsect_158032058622.html
subsect_158031974255.html
subsect_158039667024.html
chapter_1542656608.html
section_1542656620.html
section_161964252762.html
chapter_1558548485.html
section_1558548493.html
section_161964200881.html
section_162273124049.html
article_160216486846.html
section_160216528415.html
section_160216498405.html
section_160216499842.html
section_160216501258.html
section_160337298977.html
section_160216636819.html
chapter_1529417663.html
section_N29714

section_4570007314.html
section_4568643476.html
section_4570011199.html
section_4570023326.html
section_4570024634.html
section_4570029068.html
section_4570035686.html
section_4570036567.html
section_4570039432.html
section_4570046360.html
section_161520605618.html
section_4568643764.html
chapter_N463253.html
section_N463422.html
section_N464443.html
section_N464759.html
section_N464943.html
section_N465450.html
chapter_N3924743.html
book_4607606682.html
chapter_4604099319.html
section_4577281258.html
section_N250284.html
section_4604115194.html
section_4604128838.html
chapter_4750134868.html
section_4750146419.html
section_4750146818.html
chapter_4470653255.html
section_4338624600.html
section_N554247.html
section_N554714.html
section_N555081.html
section_N555355.html
section_N556825.html
section_N557197.html
section_N557476.html
section_N557750.html
chapter_156596387080.html
section_157044103464.html
section_156597280900.html
section_156597435833.html
section_0505041319.html
section_

section_N337869.html
section_N338607.html
section_N338747.html
section_N338932.html
article_157052700838.html
section_164198152941.html
article_164128969835.html
article_164129209968.html
article_164129223167.html
section_N339082.html
section_1511800348.html
section_N339259.html
section_N339589.html
section_158453872555.html
section_164787364194.html
section_N474404.html
section_N474479.html
section_4323290738.html
section_N295370.html
subsect_161583999334.html
subsect_161584119542.html
section_164208316196.html
section_N133606.html
section_N133684.html
section_N133975.html
section_N134125.html
section_N134399.html
section_4320707520.html
section_4307693953.html
section_N588325.html
section_N474793.html
section_N474926.html
section_N474536.html
section_4180852421.html
chapter_1492022011.html
section_1493326489.html
section_N485109.html
bridgehead_1493644302.html
section_N487532.html
section_N487604.html
bridgehead_N487704.html
bridgehead_1493172254.html
section_1548704486.html
section_

...

SDFxml_1396551772.html
SDFxml_2905903542.html
SDFxml_3920412268.html
SDFxml_3072813389.html
SDFxml_2135359836.html
SDFxml_143509726.html
SDFxml_851294353.html
SDFxml_2365274350.html
SDFxml_2976316878.html
SDFxml_3293710780.html
SDFxml_4060707650.html
SDFxml_1780664970.html
SDFxml_1455857744.html
SDFxml_3099417013.html
SDFxml_3658052468.html
SDFxml_3167573693.html
SDFxml_2903077041.html
SDFxml_1352105821.html
SDFxml_2018173199.html
SDFxml_662691214.html
SDFxml_3904320280.html
SDFxml_3829535848.html
SDFxml_2561226792.html
SDFxml_2924255453.html
SDFxml_1255232241.html
SDFxml_1359264582.html
SDFxml_3704365369.html
SDFxml_631077762.html
SDFxml_622684221.html
SDFxml_618240401.html
SDFxml_753653971.html
SDFxml_3508126487.html
SDFxml_308834083.html
SDFxml_1838671876.html
SDFxml_2669272545.html
SDFxml_2481711110.html
SDFxml_3914685257.html
SDFxml_570714255.html
SDFxml_465593907.html
SDFxml_1968817270.html
SDFxml_1970753821.html
SDFxml_761117417.html
SDFxml_2561124994.html
SDFxml_3725477590.html

section_N3448524.html
chapter_N3448700.html
section_N3448861.html
section_N3449174.html
section_4488980213.html
section_4738049812.html
section_4738050787.html
section_4738051114.html
chapter_N3452524.html
section_N3452691.html
section_N3452954.html
section_N3458179.html
section_N3460474.html
section_N3471338.html
chapter_N3477815.html
section_N3478008.html
section_N3480490.html
section_N3480635.html
section_N3480855.html
section_N3481360.html
section_N3481947.html
section_N3484955.html
section_N3485424.html
section_N3485891.html
section_N3486046.html
section_N3486552.html
section_N3488543.html
section_158142198604.html
section_N3489077.html
section_N3489563.html
section_N3489718.html
section_N3491080.html
section_N3492280.html
section_N3493817.html
section_N3494684.html
section_N3497592.html
section_158142272877.html
section_N3498308.html
section_N3499748.html
section_N3500304.html
section_N3503649.html
section_N3504236.html
section_N3508018.html
section_N3508536.html
section_N3513702

section_4268566454.html
section_158143050776.html
section_158143122041.html
section_N3772486.html
section_N3772873.html
section_N3773233.html
section_N3773607.html
section_N3773949.html
section_N3774307.html
section_N3774667.html
section_N3775025.html
chapter_N3775421.html
section_N3775730.html
section_N3778661.html
section_N3778995.html
section_N3779329.html
section_N3779662.html
section_N3779979.html
section_N3780297.html
section_N3780619.html
section_3808757070.html
section_3808895636.html
section_3808759336.html
section_3808925990.html
section_3808927740.html
section_N3781020.html
section_N3781338.html
section_N3781656.html
section_N3782125.html
chapter_N3782560.html
section_N3784054.html
section_4443167525.html
section_4443172233.html
section_4443156951.html
section_N3784601.html
chapter_4599346578.html
section_4599373137.html
section_N3744760.html
section_N3745415.html
section_4610708264.html
chapter_4177763939.html
section_4177768855.html
section_N3713653.html
section_4349721305

section_4872749035.html
section_1487949551.html
section_1487953889.html
section_4872781772.html
section_156398968105.html
part_159078999530.html
chapter_159077938079.html
section_159744937231.html
section_159078122846.html
section_159078131165.html
section_159078138928.html
section_159078231054.html
section_159078232795.html
section_159078233559.html
section_159078229220.html
section_159078146146.html
section_159534492999.html
chapter_159078912850.html
section_159528175227.html
section_160339556833.html
section_159528246271.html
section_159528281588.html
section_157867783812.html
section_160339768850.html
section_159528925795.html
section_159529293181.html
section_159536094519.html
section_159536224927.html
section_159536224935.html
section_159536352376.html
section_160339797342.html
section_159712552166.html
section_159536352393.html
part_156294612382.html
chapter_1539374142.html
section_159710574221.html
section_1539374153.html
section_1539375521.html
section_1539375486.html
section_

section_160146897977.html
subsect_160154427786.html
chapter_4344189507.html
section_4084991689.html
section_4084992363.html
section_4719348078.html
section_4719349429.html
section_4084993071.html
section_4718581080.html
section_4718581627.html
section_4718583675.html
chapter_157830168935.html
bridgehead_159421062967.html
section_157830254460.html
section_157830256115.html
section_159843168875.html
section_157830272851.html
chapter_1526393632.html
section_1526393714.html
section_1526393726.html
section_1526393807.html
section_1541408630.html
section_0920010307.html
section_0920010337.html
section_1556609508.html
bridgehead_1556610528.html
bridgehead_1556610888.html
bridgehead_1556611218.html
bridgehead_159775319391.html
bridgehead_1556611898.html
bridgehead_1556612021.html
bridgehead_1556612314.html
subsect_1556612642.html
section_1541420619.html
section_1540548990.html
section_1540549110.html
subsect_1540549203.html
subsect_1540549220.html
section_1542793488.html
subsect_1556614164.htm

In [119]:
len(text_data)

17935

In [122]:
df["text_data"] = text_data

In [124]:
df.to_csv("data_with_text.csv")

In [32]:
for item in sp.find_all("p"):
    print(item.text)

You must assign one or more roles to each employee to whom you want to provide access to NetSuite. Each role is a set of associated permissions that determine the data assigned users can see and the tasks they can perform.
You assign roles to an employee on the Roles subtab of the Employee record's Access tab.
If you have the Effective Dating feature enabled, the Role field is not effective-dated. If you change an employeeâs role settings, as part of an effective-dated change, this change is immediate.
For general information about roles and NetSuite access, see NetSuite Access Overview.
NetSuite provides many standard roles with predefined permissions. To view a list of the standard roles and permissions, see Standard Roles Permissions Table.
You can customize the standard roles to fit the unique needs of your business. For information about how to customize a role, see Customizing or Creating NetSuite Roles.
If you use the Global Permissions feature, in addition to assigning roles 

In [79]:
spans[2].text

'\nSneak Peeks\n'

In [78]:
paragraphs[0].text

'\nGeneral Notices\n'

Previous JavaScript must be enabled to correctly display this content Additional Resources Frequently Asked Questions  See the questions and answers below for information on Campaign Assistant. No. Campaign Assistant is designed for event-related marketing emails only. This SuiteApp is available for free in any NetSuite edition as long as the prerequisite feature Marketing Automation is available. Go to Setup > Company > Enable Features and click the CRM subtab to determine if this is available in your account. Before you can use Campaign Assistant, the Marketing Automation and Subscription Categories features must be enabled. Go to Setup > Company > Enable Features and click the CRM subtab to enable these features. Yes, your campaign can be used for several different types of events depending on what has been defined in your event details. Yes. There are two types of reports available, Campaign Response Summary and Campaign Response Detail, which provide information regarding the number of recipients who opened the email, clicked through the links, responded, purchased, and so on. To view these reports, go to Reports > Marketing > Campaign Response. Yes. NetSuite has available reports that could help you track customer responses. To view these reports, go to Reports > Marketing > Campaign Response. In Step 5: Entering Event and Schedule Details, in the Schedule Information section, you must select Execute as the status of your event, then choose a date and time to determine when your event will occur. Setting the event status toÂ ExecuteÂ tells NetSuite to generate and send your email messages at the date and time you choose. Yes, there is integration between NetSuite CRM and Constant Contact. You can use external editors to edit existing HTML templates. However, it is recommended to read Best Practices for Designing HTML Templates for Use with Marketing Campaigns and Email Templates first. Yes. When you create a new marketing template and upload the HTML file, check the Send Email Preview box and enter your email address. Once you save the template, you will receive the preview through email. Yes. Click the Source editing icon on the top left of the text editor to switch to HTML mode. Yes. In Step 4: Event Template, select File, then select -New- from the list. A popup window will appear to let you select your file from your computer. No. The Campaign Assistant is for email campaigns only. To create phone call campaigns, you must create a campaign record. Go to Lists > Marketing > Marketing Campaigns > New. On the Events subtab, click Other Events. On the Channel column, select Phone from the list. You can create new offers in Setup > Marketing > Campaign Management > Offers > New. To create a promotion code, you must first enable the Promotion Codes feature in Setup > Company > Enable Features. On the Transactions subtab, check the Promotion Codes box. Promotion codes can be created in Lists > Marketing > Promotions > New. Campaign events are used when a single marketing campaign has several distinct phases or channels. Events are tracked on the Email, Direct Mail, and Other Events subtabs under the Events tab on campaign records. Campaign events are used for scheduled aspects of a campaign, like email blasts. You can schedule the time and date of a campaign event. The Administrator role can access the Campaign Assistant without any additional permissions. The following roles also have access to the SuiteApp upon installation: Marketing Administrator Marketing Assistant Marketing Manager The term "verticals" in business refers to a vertical market. Basically, a vertical market is one where businesses and marketers cater to the needs of a specific group of people within an industry. This concept is in contrast to a horizontal market, where the focus is diverted to a large amount of people regardless of industry. To create a static group, you can add any member with same record type in the group individually. To create a dynamic group, the members are determined by the results of the saved search you selected. For more information, see Creating Email Campaigns. You can create a saved search for your campaign in Lists > Search > Saved Searches > New. Click Campaign and select your criteria, then click Save to run the search. For more information, see Saved Searches. There is no specific criterion in a saved search that tells whether the person or record has already been included in a previous campaign. You can try to create a saved search with the following criteria: Campaign Event: none of = select campaigns as needed. Click Save to save the search definitions and run the search. For more information, see Defining a Saved Search. No. You can create groups in Lists > Relationships > Groups > New. No, test cells are currently supported in the standard marketing campaign only. Customers, vendors, partners, and employees who are currently subscribed can receive email you generate in bulk through campaigns. Subscription data is pre-set according to your marketing preferences. To view and edit them, go to Setup > Marketing > Marketing Preferences. You can also change it manually for each record or through mass updates. Yes. You must first create a saved search for customers who abandoned the cart, then use the saved search to make your dynamic group. Go to Lists > Search > Saved Searches > New. Select Shopping Cart. On the Criteria subtab, select Date Created. On the popup window: Set Date Created to: within and custom Enter a date in the To and From fields. Click Set. On the Results subtab: In the Sort By field, select Internal ID from the list. On the Columns tab, select Shopper Field: ID from the list. Click Add. Click Preview to review the search results without saving the search definitions or Save to save the search definitions and run the search. General Notices